# 🎤 Whisper + MarianMT + Coqui TTS Pipeline on Paperspace

This notebook demonstrates a full speech translation pipeline:
1. Audio input using Whisper
2. Translation using MarianMT (HuggingFace)
3. Speech synthesis using Coqui TTS

✅ Offline, GPU-supported, open-source, and works on Paperspace.

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers sentencepiece
!pip install TTS
!pip install gradio torchaudio pydub


In [ ]:
import whisper
import torch
from transformers import MarianMTModel, MarianTokenizer
from TTS.api import TTS
import gradio as gr


In [ ]:
# Load Whisper model
whisper_model = whisper.load_model("base")  # or 'small', 'medium', 'large'

# Load MarianMT model and tokenizer (English to Hindi example)
translation_model_name = "Helsinki-NLP/opus-mt-en-hi"
translator_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translator_model = MarianMTModel.from_pretrained(translation_model_name)

# Load Coqui TTS model
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=torch.cuda.is_available())


In [ ]:
def translate_text(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**tokens)
    return tokenizer.decode(translated[0], skip_special_tokens=True)


In [ ]:
def pipeline(audio_path):
    # 1. Transcribe
    result = whisper_model.transcribe(audio_path)
    english_text = result['text']

    # 2. Translate
    translated_text = translate_text(english_text, translator_model, translator_tokenizer)

    # 3. TTS
    output_path = "output.wav"
    tts.tts_to_file(text=translated_text, file_path=output_path)

    return english_text, translated_text, output_path


In [ ]:
gr.Interface(
    fn=pipeline,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs=["text", "text", gr.Audio(type="filepath")],
    title="🎤 Whisper + MarianMT + Coqui TTS",
    description="Upload audio, get translated speech back!"
).launch()
